<a href="https://colab.research.google.com/github/tonytani37/stats/blob/main/%E3%82%B9%E3%82%BF%E3%83%83%E3%83%84%E3%83%87%E3%83%BC%E3%82%BF%E4%BD%9C%E6%88%90.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### まずはPDFから画像へ変換します

In [ ]:
# PDFから画像へ変換するためのライブラリ、ツールをインストールする
!apt-get install poppler-utils 
!pip install pdf2image

In [ ]:
#OCRであつかうPDFファイルを読み込む（PC上のファイルを指定してくだされ）
from google.colab import files
f = files.upload()
#読み込んだ画像のファイル名
filename = list(f)[0]

In [ ]:
# 画像データに変換
from pathlib import Path
from pdf2image import convert_from_path

# Google Colabのデフォルト環境内にimageというフォルダを作る
!mkdir /content/image/

pdf_path = Path(filename)
#outputのファイルパス（この場合は上でつくったimageフォルダを指定）
img_path=Path("/content/image/")

#変換されたjpegファイルが保存されます
convert_from_path(pdf_path, output_folder=img_path,fmt='jpeg',output_file=pdf_path.stem)


### 以上で画像変換完了。PDFに複数ページがある場合、ページごとに画像データ（ファイル名末尾が連番）が作成されるので、一つのPDFファイルから複数の画像ファイルが作成されたとしても、驚かないこと。

### ここからOCR読み込み
まずはOCR読み込み用のライブラリとツールをインストールします

In [ ]:
# Tesseract OCRの関連ライブラリーとPythonラッパーをインストール
# OCR読み込みに必要なツールをインストールします
!apt install tesseract-ocr
!apt install libtesseract-dev
!pip install pyocr

In [ ]:
 #日本語トレーニングデータのダウンロードとインストール
!curl -L https://github.com/tesseract-ocr/tessdata/raw/master/jpn.traineddata > jpn.traineddata
!cp jpn.traineddata /usr/share/tesseract-ocr/4.00/tessdata

In [137]:
#モジュールをインポート
from PIL import Image
import sys
import pyocr
import pyocr.builders

### OCR読み込みの前に、必要なツールがインストールされているか確認しませう。
うまくいかない場合は環境構築が間違っているので、最初からもう一度がんばろう

OCR tool is 'Tesseract (sh)' と表示されればオーケー

In [ ]:
#OCRが使用可能かの設定状態チェック
tools = pyocr.get_available_tools()
if len(tools) == 0:
    print("OCR tool is not found")
    sys.exit(1)

#OCRツール名を表示
tool = tools[0]
print("OCR tool is '%s'" % (tool.get_name()))

Available languages:　の次に　jpn　が入ってればオーケー（その他 osd,engなどもあるはず。もしかするとないものもあるかもしれないけど、とりあえず jpn が入っていればオーケー）

In [ ]:
#OCR対応言語を表示
langs = tool.get_available_languages()
print("Available languages: %s" % ", ".join(langs))
lang = langs[0]
print("Will use lang '%s'" % (lang))

### ここから実際のOCR読み込み処理です。
ここでは、fnameという変数にファイルのパスをセットしています。Google Colaboでは画面左側に作成されたファイルを扱うので、'/content/以降の部分に読み込みたい画像ファイル名(pathも正しく指定してね)を指定してください。ファイル名とパスの指定は、読み込みたいファイルを右クリックすると「パスをコピー」という項目があるので、そいつをクリックしてコピーし、fname = 以降に”または’のどちらかで囲って、その間にペーストすればオーケーです。
(次は複数ファイルを一括で読み込む仕掛けも考えます）

In [145]:
 #読み込んだ画像をOCRでテキスト抽出。

#読み込む画像ファイルを指定する
fname = '/content/image/20201215_X1_FF_OS_590001-6.jpg'

txt = tool.image_to_string(
    Image.open(fname),
    lang="jpn",
    builder=pyocr.builders.TextBuilder(tesseract_layout=6)
    )

# プレーに関係ありそうな文字として　"&",'Penalty','Kick-off','Extra Point','TIMEOUT','Quarter'が含まれる行だけ抽出する
lines_strip = [line for line in txt.splitlines() if ('&' in line) or ('Penalty' in line) or ('Kick-off' in line) or ('Extra Point' in line) or ('TIMEOUT' in line) or (':' in line) or ('Quarter' in line)]

### なんだか数字の読み取りがうまくいってない様子なので、マル付き数字をむりやり半角数字（str）に置き換える
うまくいっている場合にこれを動かしても（あえてマル付き数字を使っている場合以外は）なんの影響もないので、とりあえず動かしてみてください。

In [178]:
# 置き換わらないマル数字（やその他の文字）が出てきたら、conv_text辞書に追加してね。形式は辞書のkey:valueです）
conv_text = {"①":"1","②":"2","③":"3","④":"4","⑤":"5","⑥":"6","⑦":"7","⑧":"8","⑨":"9",
             "⑩":"10","⑪":"11","⑫":"12","⑬":"13","⑭":"14","⑮":"15",
             "⑯":"16","⑰":"17","⑱":"18","⑲":"19","⑳":"20"}

def num_conv(line):
    for ct in conv_text:
        line = line.replace(ct,conv_text[ct])
    return line

lines_strip_after =  [''.join(a.split(' ')) for a in [num_conv(line) for line in lines_strip]]

# 変換した結果を試し打ち
for a in lines_strip_after[1:-2]:
    print(a)

PlaybyPlayFirstQuarter
0S35Kick-off_紅2山崎丈路キック…FF0,#9宜本潤平23yリターン(#47高橋悟)
富士通フロンティアーズ12:00
1&10-FF23MRUN#29サマジーグラント12yランR1
1&10-FF35L_RUN_#3マイケルバードソン7yラン(#23ByronBeattyJr)
2&3-FF42LPASS#3マイケルバードソン一#22岩松慰封6yパス(#21BronsonBeatty,#13岩本卓也P2
1&10-FF48MPASS#3マイケルバードソンー#9宜本潤平3yパス(#4久保颯)
2&7-049RPASS。#3マイケルパードソンパス失敗
3&7-0549RPASS#3マイケルパードソン一#81中村輝晃クラーク6yパス(#27田中雄大
TIMEOUT富士通フロンティアーズ1回目
4&1-0S43RPUNT#39吉田元紀パント…0S9,#7池井勇輝フェァアキャッチ
オービックシーガルズ9:28
1&10-089LRUN紅1JimmyLaughrea6yラン(#91斉川尚之)
2&4-0S15LRUN#29李卓29ラン(#S5竹内修平)
3&2-0S17LPASS紅1JimmyLaughrea一#85HoldenHuff5yパス(#49久下裕一朗P1
1&10-0S22し_RUN紅1JimmyLaughreaQBサック-5yロス(#91斉川尚之
2&15-0S17L_RUN#29李卓29ラン(#43井本健一朗,#23奥田凌大)
3&13-0S19RPASS紅1JimmyLaughrea一#7池井勇輝11yパス(#34樋田祥一
4&2-0S30し_PUNT#8長尾健パント…FF30,約3高津佐隼矢フェアキャッチ
富士通フロンティアーズ5:27
1&10-FF30RPASS#3マイケルパードソンつ#22岩松墓封0yパス(#6三宅剛司)-NoPlay
+Penalty0S如6パスインターフェア8y罰退x8
1.&10-FF38L_PASS#3マイケルバードソンパス失敗
2&10-FF38LPASS_#3マイケルバードソン一#87水野悠司6yパス(紅6三宅剛司
3&4-FF44MRUN#3マイケルパードソン24yランR4
1.&10-0S32M_PASS#3マイケルバードソンパス失敗
2&10-0832M_PA

### おまけ
PDFからデータ化した結果を出力したりしますよ

In [179]:
# OCR処理をした結果文を出力したいときに使う（左側にstats.txtという名前のファイルができる）

# 同じファイル名で繰り返し処理をすると上書きされるので、複数ファイルを扱うとkにはtxt_nameにファイル名を指定してね
txt_name = "stats01.txt"

from google.colab import files
import pandas as pd

# いろいろいじくってリストにしたのでpandasを使って出力する
df = pd.DataFrame(lines_strip_after)
df.to_csv(txt_name,header=False,index=False)


# 続いて作成されたファイルをダウンロードしようとします
# ダウンロードしたくない場合には以下の行をコメントにするか、保存画面が出てきたらキャンセルするとかしてね
files.download(txt_name)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>